In [ ]:
# default_exp preprocess

# preprocess

In [ ]:
# export

import pandas as pd

In [ ]:
# export

def fc_count2tpm(df,samples):
    results = df.copy()
    rate=results[samples].divide(results['Length'],axis=0)
    results[samples]=rate/rate.sum(axis=0)*1e6
    return results

def fc_count2fpkm(df,samples):
    results = df.copy()
    total_reads = results[samples].sum(axis=0)
    rate = results[samples].divide(results['Length'],axis=0)
    results[samples] = rate.divide(total_reads,axis=1) * 1e6
    return results



In [ ]:
# export

def featurecount_rename(featurecounts,clinical,prefix,*,sample_title='sample',bam_title='bam',count_title='Geneid'):
    '''
    featurecounts will use the bamfile name as the sample name. This script rename the featurecounts by the given table.
    
    :param str featurecounts: featurecounts table file
    :param str clinical: clinical table file with header
    :param str prefix: output prefix, there will be four outputs, one is count output (prefix_count.txt), and the other one is the rename featurecounts (prefix_featurecounts.txt) and FPKM and TPM {prefix}_featurecounts_FPKM.txt {prefix}_featurecounts_TPM.txt
    :param str sample_title: the column name of the sample name
    :param str bam_title: the column name of the bamfile name
    :param str count_title: the column name used as identity in count data
    '''
    
    fc = pd.read_csv(featurecounts,sep='\t',comment='#')
    cli = pd.read_csv(clinical,comment='#')
    fc.rename(columns=cli.set_index(bam_title)[sample_title], 
                 inplace=True)
    samples=list(cli[sample_title])
    fc.to_csv(f'{prefix}_featurecounts.txt',sep='\t',index=False)
    fc[[count_title]+samples].to_csv(f'{prefix}_counts.txt',sep='\t',index=False)
    fpkm=fc_count2fpkm(fc,samples)
    tpm=fc_count2tpm(fc,samples)
    fpkm.to_csv(f'{prefix}_featurecounts_FPKM.txt',sep='\t',index=False)
    tpm.to_csv(f'{prefix}_featurecounts_TPM.txt',sep='\t',index=False)

In [ ]:
featurecounts='tests/fc.txt'
clinical='tests/clinical.txt'
prefix='tests/rename'
sample_title='sample'
bam_title='starbam'
count_title='gene_name'
featurecount_rename(featurecounts,clinical,prefix,sample_title=sample_title,bam_title=bam_title,count_title=count_title)

In [ ]:
!head tests/fc.txt tests/clinical.txt tests/rename*

==> tests/fc.txt <==
#Program:featureCounts	v2.0.1;	Command:"featureCounts"	"-T"	"12"	"-p"	"-t"	"exon"	"-g"	"gene_id"	"-M"	"--ignoreDup"	"--extraAttributes"	"gene_name,gene_biotype"	"-a"	"/aegis/database/mus_musculus/99/transcriptome.gtf"	"-o"	"pair-end/results/RNA/merged/star_featurecounts.txt"	"--tmpDir"	"pair-end/results/RNA/tmp"	"pair-end/results/RNA/SRR12678525/align_data/SRR12678525_STAR_Aligned.sortedByCoord.out.bam"	"pair-end/results/RNA/SRR12678526/align_data/SRR12678526_STAR_Aligned.sortedByCoord.out.bam"	"pair-end/results/RNA/SRR12678527/align_data/SRR12678527_STAR_Aligned.sortedByCoord.out.bam"	"pair-end/results/RNA/SRR12678528/align_data/SRR12678528_STAR_Aligned.sortedByCoord.out.bam"	"pair-end/results/RNA/SRR12678529/align_data/SRR12678529_STAR_Aligned.sortedByCoord.out.bam"	"pair-end/results/RNA/SRR12678530/align_data/SRR12678530_STAR_Aligned.sortedByCoord.out.bam"	
Geneid	Chr	Start	End	Strand	Length	gene_name	gene_biotype	pair-end/results/RNA/SRR12678525/align_data/SRR1

In [ ]:
df=pd.read_csv(prefix+'_featurecounts.txt',sep='\t')
samples=['SRR12678525', 'SRR12678526', 'SRR12678527', 'SRR12678528', 'SRR12678529', 'SRR12678530']

In [ ]:
fc_count2tpm(df,samples)

,Geneid,Chr,Start,End,Strand,Length,gene_name,gene_biotype,SRR12678525,SRR12678526,SRR12678527,SRR12678528,SRR12678529,SRR12678530
0,ENSMUSG00000102693,1,3073253,3074322,+,1070,4933401J01Rik,TEC,0.0,0.0,0.0,0.0,0.0,0.0
1,ENSMUSG00000064842,1,3102016,3102125,+,110,Gm26206,snRNA,0.0,0.0,0.0,0.0,0.0,0.0
2,ENSMUSG00000051951,1;1;1;1;1;1;1,3205901;3206523;3213439;3213609;3214482;342170...,3207317;3207317;3215632;3216344;3216968;342190...,-;-;-;-;-;-;-,6094,Xkr4,protein_coding,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0
3,ENSMUSG00000102851,1,3252757,3253236,+,480,Gm18956,processed_pseudogene,0.0,0.0,0.0,0.0,0.0,0.0
4,ENSMUSG00000103377,1,3365731,3368549,-,2819,Gm37180,TEC,0.0,0.0,0.0,0.0,0.0,0.0
5,ENSMUSG00000104017,1,3375556,3377788,-,2233,Gm37363,TEC,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
fc_count2fpkm(df,samples)

,Geneid,Chr,Start,End,Strand,Length,gene_name,gene_biotype,SRR12678525,SRR12678526,SRR12678527,SRR12678528,SRR12678529,SRR12678530
0,ENSMUSG00000102693,1,3073253,3074322,+,1070,4933401J01Rik,TEC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,ENSMUSG00000064842,1,3102016,3102125,+,110,Gm26206,snRNA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,ENSMUSG00000051951,1;1;1;1;1;1;1,3205901;3206523;3213439;3213609;3214482;342170...,3207317;3207317;3215632;3216344;3216968;342190...,-;-;-;-;-;-;-,6094,Xkr4,protein_coding,164.095832,164.095832,164.095832,164.095832,164.095832,164.095832
3,ENSMUSG00000102851,1,3252757,3253236,+,480,Gm18956,processed_pseudogene,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,ENSMUSG00000103377,1,3365731,3368549,-,2819,Gm37180,TEC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,ENSMUSG00000104017,1,3375556,3377788,-,2233,Gm37363,TEC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
